# GPU for pytorch

In [1]:
import torch

In [2]:
torch.cuda.is_available()

True

In [3]:
torch.cuda.current_device()

0

In [4]:
torch.cuda.get_device_name()

'Tesla T4'

In [5]:
torch.cuda.memory_allocated()

0

In [6]:
torch.cuda.memory_cached()

/usr/local/lib/python3.10/dist-packages/torch/cuda/memory.py:416: FutureWarning: torch.cuda.memory_cached has been renamed to torch.cuda.memory_reserved
  warnings.warn(


0

In [7]:
a = torch.FloatTensor([1.0, 2.0])

In [8]:
a

tensor([1., 2.])

In [9]:
a.device

device(type='cpu')

## Use GPU and CUDA

In [10]:
a = torch.FloatTensor([1.0, 2.0]).cuda()

In [11]:
a.device

device(type='cuda', index=0)

In [12]:
torch.cuda.memory_allocated()

512

## Imports

In [13]:
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

## Creating Model

In [58]:
class Model(nn.Module):

  def __init__(self, in_features=4, h1=8, h2=9, out_features=3):
    super().__init__()
    self.fc1 = nn.Linear(in_features, h1)    # input leayer
    self.fc2 = nn.Linear(h1, h2)             # hidden layer
    self.out = nn.Linear(h2, out_features)   # output layer

  def forward(self, X):
    X = F.relu(self.fc1(X))
    X = F.relu(self.fc2(X))
    X = self.out(X)
    return F.log_softmax(X, dim=1)

In [59]:
model = Model()

In [60]:
# our model is not on gpu
next(model.parameters()).is_cuda

False

### store on gpu

In [61]:
gpumodel = model.cuda()

In [62]:
next(gpumodel.parameters()).is_cuda

True

#### IRIS Dataset

In [63]:
from sklearn.datasets import load_iris

In [64]:
iris = load_iris()
df = pd.DataFrame(iris.data, columns = iris.feature_names)

In [65]:
df['target'] = iris.target

In [66]:
df.head(3)

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0


In [67]:
X = df.drop("target", axis=1).values

In [68]:
y = df["target"].values

In [69]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [91]:
# gpu and cuda
X_train = torch.Tensor(X_train).cuda()

In [92]:
X_test = torch.Tensor(X_test).cuda()

In [93]:
print(X_train.dtype)

torch.float32


In [94]:
y_train = torch.Tensor(y_train).cuda()

In [95]:
y_test = torch.Tensor(y_test).cuda()

In [96]:
train_loader = DataLoader(X_train, batch_size=60, shuffle=True, pin_memory=True)
test_loader = DataLoader(X_test, batch_size=60, shuffle=False, pin_memory=True)

In [97]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

## Train Model

In [99]:
import time
start_time = time.time()
epochs = 100
losses = []

for i in range(epochs):

  y_pred = gpumodel.forward(X_train)

  y_train = y_train.long()
  loss = criterion(y_pred, y_train)
  losses.append(loss)

  if i%10 == 0:
    print(f"Epoch: {i} Loss: {loss.item()}")

  optimizer.zero_grad()
  loss.backward()
  optimizer.step()



end = time.time() - start_time
print(f"TOTAL TIME: ", {end})


Epoch: 0 Loss: 1.2954095602035522
Epoch: 10 Loss: 1.0326414108276367
Epoch: 20 Loss: 0.8006496429443359
Epoch: 30 Loss: 0.5559664368629456
Epoch: 40 Loss: 0.4349114000797272
Epoch: 50 Loss: 0.3544715940952301
Epoch: 60 Loss: 0.25563254952430725
Epoch: 70 Loss: 0.1585385501384735
Epoch: 80 Loss: 0.09940329939126968
Epoch: 90 Loss: 0.07476428896188736
TOTAL TIME:  {0.17632389068603516}


In [102]:
x = torch.tensor([1,2])

In [103]:
x.dtype

torch.int64